In [1]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install kss
!pip install okt
!pip install konlpy
!pip install gluonnlp
!pip install mxnet
!pip install sentence-transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ooajtna4
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ooajtna4
     |████████████████████████████████| 132 kB 7.8 MB/s 
     |████████████████████████████████| 344 kB 59.8 MB/s 
     |████████████████████████████████| 49.1 MB 163 kB/s 
     |████████████████████████████████| 4.5 MB 48.9 MB/s 
     |████████████████████████████████| 1.2 MB 48.7 MB/s 
     |████████████████████████████████| 4.2 MB 49.8 MB/s 
     |████████████████████████████████| 6.6 MB 48.0 MB/s 
     |████████████████████████████████| 596 kB 56.0 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 8.8 MB 52.0 MB/s 
     |████████

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import sys

sys.path.append('/content/drive/MyDrive/ML_Train/Backend_ML')

# from model import Model
from sentiment.sentiment_model import get_sentiment_model_class
from keyword_extractor.keyword_model import get_keyword_model_class

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [5]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
from sklearn.model_selection import train_test_split
                                          

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
import torch
import torch.nn
import itertools
import random
import numpy as np
from tqdm.notebook import tqdm
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
import os
import yaml
import gluonnlp as nlp
from konlpy.tag import Okt
from kss import split_sentences
import gluonnlp as nlp

cfg = {}

with open('/content/drive/MyDrive/ML_Train/Backend_ML/sentiment/configs/cfg.yaml') as f:
    sentiment_cfg = yaml.load(f, Loader=yaml.FullLoader)

with open('/content/drive/MyDrive/ML_Train/Backend_ML/keyword_extractor/configs/cfg.yaml') as f:
    keyword_cfg = yaml.load(f, Loader=yaml.FullLoader)

# mm = Model(cfg)

# x = '솔직히 말해 그다지 예쁜 여자아이는 아니다.눈에 띄는 데가 있는 것도 아니다.멋진 옷을 입고 있는 것도 아니다.머리카락 뒤쪽에는 나쁜 잠버릇이 끈질기게 달라붙어 있고,나이도 적지 않다.벌써 서른 살에 가까울테니까.엄밀히 말하면 여자아이라고 할 수도 없으리라.그럼에도 불구하고, 나는 50미터 떨어진 곳에서부터그녀를 알아볼 정도다.그녀는 내게 있어서 100%의 여자아이이기 때문이다.그녀의 모습을 목격하는 순간부터내 가슴은 땅울림처럼 떨리고,입안은 사막처럼 바싹 말라버린다.'



sentiment_model_class = get_sentiment_model_class(sentiment_cfg['model_name'])
sentiment_model = sentiment_model_class(sentiment_cfg)

keyword_model_class = get_keyword_model_class(keyword_cfg['model_name'])
keyword_model = keyword_model_class(keyword_cfg)

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
content = '뭐해 밥'

score, emotion = sentiment_model.predict(content)
tmp_keywords = keyword_model.predict(content)

hashtag = {}
for i in range(len(tmp_keywords)):
    hashtag['hashtag_{0}'.format(i)] = tmp_keywords[i]

result = {'emotion' : emotion}
result.update(hashtag)
result.update(score)

result

[Korean Sentence Splitter]: Initializing Pynori...


문장 별 score


{'anger_score': 0.48,
 'disgust_score': 0.4,
 'emotion': '분노',
 'fear_score': 0.03,
 'happy_score': 0.01,
 'hashtag_0': None,
 'hashtag_1': None,
 'hashtag_2': None,
 'neutral_score': 0.04,
 'sad_score': 0.02,
 'suprise_score': 0.02}